In [1]:
import pandas as pd

import torch
import torch.nn as nn
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from helper_functions import normalize, accuracy_fnn
from torch.utils.data import DataLoader, TensorDataset 
from CNN import *
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device =', device)

c:\Program Files\Python311\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


device = cpu


In [2]:
from DataLoader import Loaders


df = Loaders(batch_size = 16 ,normalization= False,file_path='future_clean/future-1/')

In [3]:
df.train , df.train.shape

(array([[9.03015898e-05, 1.76219764e-04, 4.24878308e-05, ...,
         1.39140291e-04, 1.15248236e-04, 0.00000000e+00],
        [4.14241367e-04, 4.44046920e-04, 9.30094466e-05, ...,
         2.60125729e-04, 4.13253991e-04, 0.00000000e+00],
        [4.59137838e-04, 5.69552940e-04, 8.77583152e-05, ...,
         3.67744127e-04, 3.40097118e-04, 0.00000000e+00],
        ...,
        [2.94318656e-04, 2.63341324e-04, 3.06061993e-04, ...,
         3.47068737e-04, 8.35544124e-05, 1.00000000e+00],
        [6.44881875e-05, 8.31654397e-05, 9.84251528e-05, ...,
         1.09376248e-04, 4.64972436e-05, 1.00000000e+00],
        [3.25540568e-05, 3.51450253e-05, 3.95705320e-05, ...,
         4.12012268e-05, 3.38705795e-05, 1.00000000e+00]]),
 (24320, 9))

In [4]:
model_ = CNNModel(in_channels=8 )

In [5]:
from timeit import default_timer as timer 
def print_train_time(start: float, end: float, device: torch.device = None):
    """Prints difference between start and end time.

    Args:
        start (float): Start time of computation (preferred in timeit format). 
        end (float): End time of computation.
        device ([type], optional): Device that compute is running on. Defaults to None.

    Returns:
        float: time between start and end in seconds (higher is longer).
    """
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time

In [6]:
def training_loop( model: torch.nn.Module,
                      data_loader_train: torch.utils.data.DataLoader,
                      data_loader_test: torch.utils.data.DataLoader,
                      loss_fn: torch.nn.Module,
                      optimizer: torch.optim.Optimizer,
                      accuracy_fn,
                      device: torch.device,
                      epochs: int
                      ,inplace=False):
        """Treina e testa um modelo PyTorch.

     Passa um modelo PyTorch de destino por meio de train_step() e test_step()
     funções para um número de épocas, treinando e testando o modelo
     no mesmo loop de época.

     Calcula, imprime e armazena métricas de avaliação.

     Argumentos:
       model: um modelo PyTorch a ser treinado e testado.
       train_dataloader: Uma instância do DataLoader para o modelo a ser treinado.
       test_dataloader: Uma instância do DataLoader para o modelo a ser testado.
       otimizador: Um otimizador PyTorch para ajudar a minimizar a função de perda.
       loss_fn: uma função de perda do PyTorch para calcular a perda em ambos os conjuntos de dados.
       epochs: Um número inteiro indicando para quantas épocas treinar.
       dispositivo: um dispositivo de destino para calcular (por exemplo, "cuda" ou "cpu").
      
     Retorna:
       Um dicionário de perda de treinamento e teste, bem como treinamento e
       testar métricas de precisão. Cada métrica tem um valor em uma lista para
       cada época.
       Na forma: {train_loss: [...],
                 train_acc: [...],
                 teste_perda: [...],
                 test_acc: [...]}
       Por exemplo, se o treinamento for epochs=2:
               {train_loss: [2.0616, 1.0537],
                 train_acc: [0,3945, 0,3945],
                 perda_teste: [1.2641, 1.5706],
                 test_acc: [0,3400, 0,2973]}
     """

        results_array_test = np.empty((epochs, 2))
        results_array_train = np.empty([epochs, 2])

        print(f"Training on {device}")
        for epoch in tqdm(range(epochs)):

            
            print(f" Epoch: {epoch}\n---------")
            # loop pelo dataloader de treino
            model.to(device)
            model.train().double()
            training_loss = 0
            training_accurary = 0
            valid_loss = 0
            for batch, (inputs, target) in enumerate(data_loader_train):
                # movendo os dados para o dispositivo de processamento
                inputs = inputs.to(device).double()
                inputs = inputs.unsqueeze(2)
                # target = target.unsqueeze(1)
                
                # fazendo as previsões
                output = model(inputs.double())

                # calculando a perda
                loss = loss_fn(output, target.long())

                training_loss += loss.data.item()
                training_accurary += accuracy_fn(target, output.argmax(dim=1))

                # retropropagando os gradientes e atualizando os pesos
                optimizer.zero_grad()

                loss.backward()

                optimizer.step()

            training_loss /= len(data_loader_train)
            training_accurary /= len(data_loader_train)

            # imprimindo as métricas de treino a cada 50 lotes
            
            print(
                    f'Train loss: {training_loss:.5f} | Train accuracy: {training_accurary:.2f}%')

            results_array_train[epoch, 0] = "%.2f" % training_loss
            results_array_train[epoch, 1] = "%.2f" % training_accurary

            # avaliando o modelo no dataloard de teste
            # loop pelo dataloader de teste

            model.eval().double()
            valid_loss = 0
            test_loss = 0
            test_accurary = 0
            with torch.inference_mode():

                for data, target in data_loader_test:
                    # movendo os dados para o dispositivo de processamento
                    data = data.to(device)
                    target = target.to(device)
                    # target = target.unsqueeze(1)
                    
                    data = data.unsqueeze(2)
                    # 1. Forward pass
                    test_pred = model(data)
                    # 2. Calculate loss and accuracy
                    test_loss += loss_fn(test_pred, target.long())
                    valid_loss += test_loss.data.item()
                    test_accurary += accuracy_fn(target,
                                                 test_pred.argmax(dim=1))

                valid_loss /= len(data_loader_test)
                test_accurary /= len(data_loader_test)
            

            print(
                    f'Test loss: {valid_loss:.5f} | Test accuracy: {test_accurary:.2f}%')

            results_array_test[epoch, 0] = "%.2f" % valid_loss
            results_array_test[epoch, 1] = "%.2f" % test_accurary

In [7]:
loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(params=model_.parameters(), lr= 1e-02)

In [8]:
training_loop(model = model_,
                      data_loader_train= df.train_loader,
                      data_loader_test=df.test_loader,
                      loss_fn = loss_fn,
                      optimizer =optimizer,
                      accuracy_fn = accuracy_fnn,
                      device = device,
                      epochs =30
                      ,inplace=False)

Training on cpu


  0%|          | 0/30 [00:00<?, ?it/s]

 Epoch: 0
---------
Train loss: 2.70623 | Train accuracy: 44.12%
Test loss: 268.93685 | Test accuracy: 53.78%
 Epoch: 1
---------
Train loss: 2.52277 | Train accuracy: 57.80%
Test loss: 251.53803 | Test accuracy: 55.84%
 Epoch: 2
---------
Train loss: 2.37487 | Train accuracy: 63.68%
Test loss: 251.65690 | Test accuracy: 48.59%
 Epoch: 3
---------
Train loss: 2.27702 | Train accuracy: 69.26%
Test loss: 239.27022 | Test accuracy: 63.84%
 Epoch: 4
---------
Train loss: 2.21642 | Train accuracy: 72.69%
Test loss: 230.60906 | Test accuracy: 69.12%
 Epoch: 5
---------
Train loss: 2.18228 | Train accuracy: 73.78%
Test loss: 224.71362 | Test accuracy: 68.66%
 Epoch: 6
---------
Train loss: 2.16843 | Train accuracy: 74.35%
Test loss: 225.51437 | Test accuracy: 68.53%
 Epoch: 7
---------
Train loss: 2.15661 | Train accuracy: 74.75%
Test loss: 224.28968 | Test accuracy: 68.91%
 Epoch: 8
---------
Train loss: 2.14755 | Train accuracy: 75.12%
Test loss: 219.24827 | Test accuracy: 72.00%
 Epoch: 9
